### Fetch the data from CRSP and replicate the table in the first paragraph of Cochrane's "dog" paper.

| Regression | b | t | R^2(%) | sigma(bx)(%) |
| ---: | :--- | :--- | :--- | :--- |
| $R_{t+1}=a+b(D_{t}/P_{t})+e_{t+1}$ | 3.39 | 2.28 | 5.8 | 4.9 |
| $R_{t+1}-R_{t}^{f}=a+b(D_{t}/P_{t})+e_{t+1}$ | 3.83 | 2.61 | 7.4 | 5.6 |
| $D_{t+1}/D_{t}=a+b(D_{t}/P_{t})+e_{t+1}$ | 0.07 | 0.06 | 0.0001 | 0.001 |


Table 1 . Forecasting regressions. $R_{t+1}$ is the real return, deflated by the CPI, $D_{t+1} / D_t$ is real dividend growth, and $D_t / P_t$ is the dividend-price ratio of the CRSP value-weighted portfolio. $\quad R_{t+1}^f$ is the real return on three-month Treasury Bills. Small letters are logs of corresponding capital letters. Annual data, 1926-2004. $\sigma(b x)$ gives the standard deviation of the fitted value of the regression.

In [78]:
import pandas as pd
import numpy as pd
import statsmodels.formula.api as smf

In [66]:
RET = pd.read_csv('stock_return.csv')

def compounded_ret(df):
    return (1 + df['vwretd']).cumprod().iloc[-1]

RET['year'] = (RET['date'].astype(int) / 10000).astype(int)
RET_R = RET.groupby(['year', 'PERMNO']).apply(compounded_ret).reset_index().rename(columns={0:'R'})
RET_R

,year,PERMNO,R
0,1926,10006,1.098488
1,1926,10014,1.098488
2,1926,10022,1.130576
3,1926,10030,1.098488
4,1926,10049,1.098488
...,...,...,...
299322,2004,92690,1.129912
299323,2004,92807,1.129912
299324,2004,92874,1.129912
299325,2004,92930,1.129912


In [56]:
RET_D = RET.groupby(['year', 'PERMNO'])['DIVAMT'].sum().reset_index().rename(columns={'DIVAMT':'D'})
RET_D

,year,PERMNO,D
0,1926,10006,8.31250
1,1926,10014,0.00000
2,1926,10022,3.00000
3,1926,10030,6.00000
4,1926,10049,4.00000
...,...,...,...
299322,2004,92690,0.94500
299323,2004,92807,0.00000
299324,2004,92874,0.04000
299325,2004,92930,2.99917


In [54]:
RET['PRC'] = RET['PRC'].abs()
RET_P = RET.groupby(['year', 'PERMNO'])['PRC'].last().reset_index().rename(columns={'PRC':'P'})
RET_P

,year,PERMNO,P
0,1926,10006,101.5000
1,1926,10014,1.3125
2,1926,10022,56.0000
3,1926,10030,137.5000
4,1926,10049,87.7500
...,...,...,...
299322,2004,92690,15.7400
299323,2004,92807,5.1700
299324,2004,92874,12.7100
299325,2004,92930,39.5900


In [64]:
INF = pd.read_csv('bond_and_inflation.csv')

INF['year'] = (INF['caldt'].astype(int) / 10000).astype(int)
INF = INF[['year', 't90ret', 'cpiret']]
INF = INF.rename(columns={'t90ret' : 'Rf', 'cpiret' : 'cpi'})
INF

,year,Rf,cpi
0,1926,0.036026,-0.011174
1,1927,0.030917,-0.022598
2,1928,0.044427,-0.011561
3,1929,0.049219,0.005848
4,1930,0.027487,-0.063953
...,...,...,...
74,2000,0.061522,0.033869
75,2001,0.044809,0.015517
76,2002,0.017951,0.023769
77,2003,0.011604,0.018794


In [91]:
DATA = RET_R
DATA = DATA.merge(RET_P, on=['year', 'PERMNO'])
DATA = DATA.merge(RET_D, on=['year', 'PERMNO'])
DATA = DATA.merge(INF, on=['year'])
DATA['RR'] = DATA['R'] - DATA['cpi']

DATA = DATA.sort_values(by=['PERMNO','year'])
DATA['RR1'] = DATA.groupby(['PERMNO'])['RR'].shift(-1)
DATA['D1'] = DATA.groupby(['PERMNO'])['D'].shift(-1)

DATA['DP'] = DATA['D']/DATA['P']
DATA['RR1Rf'] = DATA['RR1']-DATA['Rf']
DATA['DD1'] = DATA['D1']/DATA['D']
DATA

,year,PERMNO,R,P,D,Rf,cpi,RR,RR1,D1,DP,RR1Rf,DD1
131750,1985,10000,1.043061,NaN,0.00,0.085266,0.037986,1.005075,1.144650,0.00,NaN,1.059384,NaN
138895,1986,10000,1.155629,0.51563,0.00,0.067499,0.010979,1.144650,1.203957,0.00,0.000000,1.136458,NaN
146513,1987,10000,1.248300,0.21875,0.00,0.066396,0.044343,1.203957,NaN,NaN,0.000000,NaN,NaN
131751,1985,10001,1.043061,NaN,0.00,0.085266,0.037986,1.005075,1.144650,0.41,NaN,1.059384,inf
138896,1986,10001,1.155629,7.00000,0.41,0.067499,0.010979,1.144650,0.973970,0.42,0.058571,0.906471,1.02439
...,...,...,...,...,...,...,...,...,...,...,...,...,...
248853,1998,93316,1.222446,6.75000,0.00,0.053026,0.016119,1.206327,1.225873,0.00,0.000000,1.172847,NaN
258475,1999,93316,1.252721,12.87500,0.00,0.049227,0.026848,1.225873,1.010022,15.36,0.000000,0.960795,inf
267806,2000,93316,1.043891,15.15625,15.36,0.061522,0.033869,1.010022,NaN,NaN,1.013443,NaN,NaN
131749,1984,93324,1.012641,0.48438,0.00,0.111154,0.039486,0.973155,1.276129,0.00,0.000000,1.164975,NaN


In [88]:
model = smf.ols("RR1 ~ DP", data=DATA)
result = model.fit()
result.params

Intercept    1.082953
DP           0.000132
dtype: float64

In [92]:
model = smf.ols("RR1Rf ~ DP", data=DATA)
result = model.fit()
result.params

Intercept    1.023554
DP           0.000145
dtype: float64

In [93]:
model = smf.ols("DD1 ~ DP", data=DATA)
result = model.fit()
result.params

Intercept   NaN
DP          NaN
dtype: float64